In [9]:
import pandas as pd
import numpy as np
import requests
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import requests
import os
import time
from selenium.webdriver.common.by import By
import re
from timeit import default_timer as timer
from difflib import SequenceMatcher
import pandas as pd
from webdriver_manager.chrome import ChromeDriverManager

def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

options = Options()
options.add_argument("--disable-extensions")
options.add_argument("--disable-gpu")
options.add_argument("--headless")
options.add_argument("--headless")  # Run in headless mode
options.add_argument("--no-sandbox")  # Bypass OS security model
options.add_argument("--disable-dev-shm-usage")  # Overcome resource problems
options.add_argument("--disable-gpu")  # Disable GPU
options.add_argument("--window-size=400,400")  # Set a fixed window size (optional but helps)
options.add_argument("--disable-software-rasterizer")  # Disable rendering components
options.add_argument("--hide-scrollbars")  # Hide scrollbars


failures = pd.read_csv("need2fix.csv")
if "Unnamed: 0" in failures.columns:
    failures.drop("Unnamed: 0", axis = 1, inplace = True)
failures

,Card,Tags
0,Circle of Protection: Blue,[]
1,Circle of Protection: Green,[]
2,Circle of Protection: Red,[]
3,Circle of Protection: White,[]
4,Fire Elemental,[]
...,...,...
520,Season of Gathering,[]
521,Season of Loss,[]
522,Season of the Bold,[]
523,Season of the Burrow,[]


In [10]:
def search(card, options):
    driver = webdriver.Chrome(ChromeDriverManager().install())
    driver.get("https://tagger.scryfall.com/")
    search_box = driver.find_element(By.CLASS_NAME, "tag-input-field")
    if ":" in card:
        search_box.send_keys(card.split(":")[0])
    else:
        search_box.send_keys(card)
    time.sleep(2)

    selecter = driver.find_element(By.CLASS_NAME, "suggestion-menu") #Finds the dropdown of suggestions
    time.sleep(2)
    
    if ":" in card:
        time.sleep(1)
    search = selecter.find_elements(By.CLASS_NAME, "suggestion-row") #Pulls all of the rows of the dropdown
    searchlist = []
    for i in search:
        searchlist.append(i.get_attribute("innerHTML")) 
    cardstrings = []
    similarlist = []
 
    try:
        for i in searchlist: 
            if card in i:
                if "#icons-card" in i: #Find the value of the dropdown that is a specific card
                    i = i.split(">")
                    for j in i:
                        if card in j:
                            cardstrings.append(j.replace(" <!----",""))
        
        for i in cardstrings:
            sim = similar(card, i)
            similarlist.append(sim)

    except:
        time.sleep(4)
        search = selecter.find_elements(By.CLASS_NAME, "suggestion-row") #Pulls all of the rows of the dropdown
        for i in search:
            searchlist.append(i.get_attribute("innerHTML")) 
        for i in searchlist: 
            if card in i:
                if "#icons-card" in i: #Find the value of the dropdown that is a specific card
                    i = i.split(">")
                    for j in i:
                        if card in j:
                            cardstrings.append(j.replace(" <!----",""))
        
        for i in cardstrings:
            sim = similar(card, i)
            similarlist.append(sim)

    
    cardsearch = similarlist.index(max(similarlist))
    if "#icons-tag" in searchlist[cardsearch+1]:
        selecter.find_elements(By.CLASS_NAME, "suggestion-row")[cardsearch + 2].click()
    else:
        selecter.find_elements(By.CLASS_NAME, "suggestion-row")[cardsearch + 1].click()
    time.sleep(1)


    try:
        specific = driver.find_element(By.CLASS_NAME, "card-grid-item")
        specific.click()
    except:
        pass

    time.sleep(2)
    tags = driver.find_elements(By.CLASS_NAME, "tag-row-flex")

    taglist = []
    for i in tags:
        taglist.append(i.get_attribute("innerHTML"))

    cardtags = []
    for i in taglist:
        if "tags/card/" in i:
            cardtags.append(i)
    
    tagsavelist = []
    for i in cardtags:
        if "tags/card/" in i:
            tag = i.split('"">')[-1].replace("</a>","")
            tagsavelist.append(tag)

    time.sleep(2)
    oldtags = driver.find_elements(By.CLASS_NAME, "tagging-ancestors")

    taglist = []
    for i in oldtags:
        taglist.append(i.get_attribute("innerHTML"))

    cardtags = []
    for i in taglist:
        if "tags/card/" in i:
            cardtags.append(i)

    for i in cardtags:
        if "/tags/card/" in i:
            tag = i.split(",")
            for k in tag:
                tagsavelist.append(k.split('"">')[-1].replace("</a>",""))
    
    if tagsavelist == []:
        tagsavelist.append("notags")
    
    driver.quit()

    return tagsavelist

In [11]:
def getfailures(faillist):
    results = []
    for i in faillist:
        try:
            if '"' in i:
                i = i.replace('"', '')
            cardresult = search(i, options)
            results.append(cardresult)
        except:
            results.append("-- FAILED --")
            continue

    return results

In [12]:
faillist_test = ['Vault 101: Birthday Party']
results = getfailures(faillist_test)
results

/var/folders/hf/72zhzf2j1lxcs1ht7hh8mp8c0000gn/T/ipykernel_3480/80847475.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


['-- FAILED --']

In [13]:
results = getfailures(failures["Card"])

/var/folders/hf/72zhzf2j1lxcs1ht7hh8mp8c0000gn/T/ipykernel_3480/80847475.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [14]:
results

[['artist matters',
  'bounce-self',
  'circle of protection',
  'damage prevention-you',
  'damage prevention',
  'un-design',
  'un-set mechanics'],
 '-- FAILED --',
 ['artist matters',
  'bounce-self',
  'circle of protection',
  'damage prevention-you',
  'damage prevention',
  'un-design',
  'un-set mechanics'],
 ['artist matters',
  'bounce-self',
  'circle of protection',
  'damage prevention-you',
  'damage prevention',
  'un-design',
  'un-set mechanics'],
 ['notags'],
 ['notags'],
 ['notags'],
 ['notags'],
 ['notags'],
 ['notags'],
 ['meme'],
 ['notags'],
 ['artist matters',
  'bounce-self',
  'circle of protection',
  'damage prevention-you',
  'damage prevention',
  'un-design',
  'un-set mechanics'],
 '-- FAILED --',
 ['notags'],
 ['notags'],
 ['notags'],
 ['notags'],
 ['notags'],
 ['notags'],
 ['notags'],
 ['notags'],
 ['notags'],
 ['notags'],
 ['mana sink',
  'noncreature tribal',
  'repeatable token generator',
  'sacrifice outlet-creature',
  'tribal-goblin',
  'sacrif

In [15]:
failures["Tags"] = results
failures

,Card,Tags
0,Circle of Protection: Blue,"[artist matters, bounce-self, circle of protec..."
1,Circle of Protection: Green,-- FAILED --
2,Circle of Protection: Red,"[artist matters, bounce-self, circle of protec..."
3,Circle of Protection: White,"[artist matters, bounce-self, circle of protec..."
4,Fire Elemental,[notags]
...,...,...
520,Season of Gathering,"[cycle-blb-season, draw, gives pp counters, gi..."
521,Season of Loss,"[cycle-blb-season, draw, modal, morbid, oppone..."
522,Season of the Bold,"[cast trigger-you, cycle-blb-season, delayed t..."
523,Season of the Burrow,"[cycle-blb-season, gives indestructible, indes..."


In [16]:
failures.to_csv("need2fix.csv")